# Preprocess CBS extracts

This notebook will prepare the CBS excel extracts for easy analysis.

In [2]:
import pandas as pd
import numpy as np
from siuba import *

## Accident count

In [3]:
double_lane = [11.9, 60.4]   # From Eilot junction to Ktora Junction
single_lane = [60.4, 180.5]  # Till Arava Junction

length_single_lane = single_lane[1] - single_lane[0]
length_double_lane = double_lane[1] - double_lane[0]

In [4]:

df = pd.read_excel("../data/accidents_with_casualties/file_084b50e9-6039-406d-9600-32d5122ca3d0_longer.xlsx",
    header=20, usecols="A:F", names=['location', 'year', 'total', 'fatal', 'severe', 'light'])

total_str = df.iloc[0,1]
unknown_str = df.loc[44, 'location']

df = (
    df
    .replace(total_str, np.nan)
    .replace(unknown_str, np.nan)
    .dropna(how='any', axis=0, subset=['location', 'year'])
    .fillna(0)
)
df.location = df.location.str.replace(".x", ".0")
df.location = df.location.astype(float)
df = df[df.location > 0.0]   # 0 is a summary location
# df = df >> filter(_.location >= double_lane[0], _.location <= single_lane[1])
# df = df >> mutate(place = if_else(
#     _.location >= single_lane[0], 'single lane', 'double lane'
# ))
df

/tmp/ipykernel_770039/1790370751.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  df.location = df.location.str.replace(".x", ".0")


,location,year,total,fatal,severe,light
84,0.1,2003.0,7.0,0.0,2.0,5.0
85,0.1,2004.0,8.0,0.0,1.0,7.0
86,0.1,2005.0,9.0,0.0,1.0,8.0
87,0.1,2006.0,5.0,0.0,1.0,4.0
88,0.1,2007.0,6.0,0.0,0.0,6.0
...,...,...,...,...,...,...
23833,476.0,2011.0,1.0,0.0,0.0,1.0
23835,476.2,2009.0,1.0,1.0,0.0,0.0
23836,476.2,2019.0,1.0,0.0,1.0,0.0
23838,476.6,2015.0,1.0,0.0,0.0,1.0


In [5]:
df.to_parquet("../data/preprocessed/accident_counts_2003-2022.parquet")
df.to_csv("../data/preprocessed/accident_counts_2003-2022.csv", index=False)


In [6]:
! head ../data/preprocessed/accident_counts_2003-2022.csv

location,year,total,fatal,severe,light
0.1,2003.0,7.0,0.0,2.0,5.0
0.1,2004.0,8.0,0.0,1.0,7.0
0.1,2005.0,9.0,0.0,1.0,8.0
0.1,2006.0,5.0,0.0,1.0,4.0
0.1,2007.0,6.0,0.0,0.0,6.0
0.1,2008.0,3.0,0.0,0.0,3.0
0.1,2009.0,10.0,0.0,0.0,10.0
0.1,2010.0,4.0,0.0,0.0,4.0
0.1,2011.0,7.0,0.0,2.0,5.0


## Casualty count

In [7]:

df_casualties = pd.read_excel("../data/casualties_in_accidents/file_5c6cf05f-2fa2-4ffb-b06a-a721caca2d1e_more_years_real.xlsx",
    header=22, usecols="A:F", names=['year', 'location', 'total', 'killed', 'severe_injury', 'light_injury'])

total_str = df_casualties.iloc[0, 0]
unknown_str = df_casualties.iloc[0, 1]
df_casualties = (
    df_casualties
    .replace(total_str, np.nan)
    .replace(unknown_str, np.nan)
    .dropna(how='any', axis=0, subset=['location', 'year'])
    .fillna(0)
)
df_casualties.location = df_casualties.location.str.replace(".x", ".0")
df_casualties.location = df_casualties.location.astype(float)
df_casualties

/tmp/ipykernel_770039/3863994695.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  df_casualties.location = df_casualties.location.str.replace(".x", ".0")


,year,location,total,killed,severe_injury,light_injury
820,2003.0,14.6,4,0.0,0.0,4.0
821,2003.0,16.5,4,0.0,0.0,4.0
822,2003.0,24.3,11,0.0,0.0,11.0
823,2003.0,39.9,4,0.0,0.0,4.0
824,2003.0,43.0,27,1.0,4.0,22.0
...,...,...,...,...,...,...
2271,2022.0,436.3,6,0.0,0.0,6.0
2272,2022.0,441.2,1,0.0,0.0,1.0
2273,2022.0,443.6,6,0.0,0.0,6.0
2274,2022.0,450.5,4,0.0,0.0,4.0


In [8]:
df_casualties.to_parquet("../data/preprocessed/casualty_counts_2003-2022.parquet")
df_casualties.to_csv("../data/preprocessed/casualty_counts_2003-2022.csv", index=False)

In [9]:
! head ../data/preprocessed/casualty_counts_2003-2022.csv

year,location,total,killed,severe_injury,light_injury
2003.0,14.6,4,0.0,0.0,4.0
2003.0,16.5,4,0.0,0.0,4.0
2003.0,24.3,11,0.0,0.0,11.0
2003.0,39.9,4,0.0,0.0,4.0
2003.0,43.0,27,1.0,4.0,22.0
2003.0,45.1,2,1.0,0.0,1.0
2003.0,47.5,4,0.0,0.0,4.0
2003.0,48.7,4,0.0,0.0,4.0
2003.0,51.0,9,1.0,4.0,4.0


# Preprocess new excel table from map generator

In [18]:
from pathlib import Path

In [19]:
km_mapping = {
    109: 12.4,
    1123: 54,
    1125: 57.9,
    40: 60.4,
    1127: 60.4,
    13: 97.6,
    227: 158.2,
    25: 180.5,
    31: 205.3,
    1: 287.1,
    449: 307.5,
    505: 319.6
}

In [20]:
files = Path("../data").glob("*_maps*.xlsx")
dfs = []
for file in files:
    print(file)
    road_num = int(file.as_posix().split('accidents_')[1].split('_from')[0])
    df = pd.read_excel(file, header=1, names=
        ['year', 'month', 'day of week', 'day/night', 'accident_severity', 'accident_type', 'killed', 'severly_injured',
            'lightly_injured', 'injured_pedestrians', 'casualties_ages_0-19', 'casualties_ages_20-64', 
            'casualties_ages_65_plus', 'total_casualties', 'vehicle_count', 'drivers', 'road_type',
            'localization_quality', 'settlement', 'road1', 'location', 'road2', 'road3', 'road4'
        ])
    df.dropna(subset=['month'], inplace=True)
    df['source_road'] = road_num
    if road_num == 90:
        for k in km_mapping.keys():
            to_replace = (
                df
                >> filter(_.location.isna())
                >> filter((_.road1 == k) | (_.road2 == k))
            )
            df.loc[to_replace.index, 'location'] = km_mapping[k]

    dfs.append(df)
df = pd.concat(dfs)
df

../data/accidents_4_from_maps_km_141_to_250.xlsx
../data/accidents_6_from_maps.xlsx
../data/accidents_2_from_maps.xlsx
../data/accidents_40_from_maps.xlsx
../data/accidents_90_from_maps.xlsx
../data/accidents_4_from_maps_km_52_to_140.xlsx
../data/accidents_20_from_maps.xlsx


,year,month,day of week,day/night,accident_severity,accident_type,killed,severly_injured,lightly_injured,injured_pedestrians,...,drivers,road_type,localization_quality,settlement,road1,location,road2,road3,road4,source_road
0,2003,פברואר,שישי,לילה,קלה,התנגשות חזית באחור,0.0,0.0,2.0,0.0,...,2.0,לא-עירונית בצומת,עיגון מדויק,NaN,4.0,NaN,NaN,NaN,NaN,4
1,2003,פברואר,שלישי,לילה,קלה,התנגשות חזית בצד,0.0,0.0,2.0,0.0,...,2.0,לא-עירונית בצומת,עיגון מדויק,NaN,4.0,NaN,NaN,NaN,NaN,4
2,2003,אפריל,ראשון,לילה,קלה,התנגשות חזית בצד,0.0,0.0,1.0,0.0,...,2.0,לא-עירונית בצומת,עיגון מדויק,NaN,4.0,NaN,5720.0,NaN,NaN,4
3,2003,יולי,שבת,לילה,קלה,התנגשות חזית בצד,0.0,0.0,2.0,0.0,...,2.0,לא-עירונית בצומת,עיגון מדויק,NaN,4.0,NaN,85.0,NaN,NaN,4
4,2003,ינואר,רביעי,לילה,קלה,התנגשות חזית באחור,0.0,0.0,3.0,0.0,...,2.0,לא-עירונית בצומת,עיגון מדויק,NaN,4.0,NaN,NaN,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3081,2022,פברואר,שבת,לילה,קלה,התנגשות חזית באחור,0.0,0.0,1.0,0.0,...,2.0,לא-עירונית לא בצומת,עיגון מדויק,NaN,20.0,7.4,NaN,NaN,NaN,20
3082,2022,פברואר,חמישי,לילה,קלה,התנגשות צד בצד,0.0,0.0,2.0,0.0,...,2.0,לא-עירונית לא בצומת,עיגון מדויק,NaN,20.0,14.0,NaN,NaN,NaN,20
3083,2022,פברואר,שני,יום,קלה,התנגשות עם עצם דומם,0.0,0.0,2.0,1.0,...,3.0,לא-עירונית לא בצומת,עיגון מדויק,NaN,20.0,11.9,NaN,NaN,NaN,20
3084,2022,מרס,ראשון,יום,קטלנית,התנגשות צד בצד,1.0,0.0,1.0,0.0,...,2.0,לא-עירונית לא בצומת,עיגון מדויק,NaN,20.0,11.2,NaN,NaN,NaN,20


In [21]:
df.source_road.value_counts()

4     9505
40    4224
20    3086
90    2601
2     2444
6     2091
Name: source_road, dtype: int64

In [22]:
from siuba import *

(
    df.groupby('source_road').accident_severity.value_counts(normalize=True).reset_index(name='severity')  >> filter(_.accident_severity=='קטלנית')
)

,source_road,accident_severity,severity
2,2,קטלנית,0.058101
5,4,קטלנית,0.034298
8,6,קטלנית,0.072692
11,20,קטלנית,0.028840
14,40,קטלנית,0.059186
17,90,קטלנית,0.073818


In [15]:
df.to_csv("../data/preprocessed/accidents_from_mapper.csv", index=False)
df.to_parquet("../data/preprocessed/accidents_from_mapper.parquet")

## Rename all variables to English for modelling

In [24]:
months = {
    'ינואר': 'January',
    'פברואר': 'February',
    'מרץ': 'March',
    'אפריל': 'April',
    'מאי': 'May',
    'יוני': 'June',
    'יולי': 'July',
    'אוגוסט': 'August',
    'ספטמבר': 'September',
    'אוקטובר': 'October',
    'נובמבר': 'November',
    'דצמבר': 'December'
}
days = {
    'ראשון': 'Sunday',
    'שני': 'Monday',
    'שלישי': 'Tuesday',
    'רביעי': 'Wednesday',
    'חמישי': 'Thursday',
    'שישי': 'Friday',
    'שבת': 'Saturday',
}
day_night = {
    'לילה': 'Night',
    'יום': 'Day'
}
accident_severity = {
    'קלה': 'Light',
    'קשה': 'Severe',
    'קטלנית': 'Fatal'
}
accident_type = {
    'התנגשות חזית בצד': 'Front to side collision',
    'התנגשות חזית באחור': 'Front to rear collision',
    'התנגשות צד בצד': 'Side to side collision',
    'התנגשות עם עצם דומם': 'Hit an object',
    'התהפכות': 'Roll over',
    'פגיעה בהולך רגל': 'Pedestrian',
    'התנגשות חזית בחזית': 'Front end collision',
    'החלקה': 'Slip',
    'אחר': 'Other',
    'ירידה מהכביש או עלייה למדרכה': 'Drive off road or on paveway',
    'התנגשות עם רכב שנעצר ללא חניה': 'Collision with stopped vehicle',
    'פגיעה בנוסע בתוך כלי הרכב': 'Hit a passenger within the vehicle'
}
df.accident_type.value_counts()

התנגשות חזית בצד                 9723
התנגשות חזית באחור               5382
התנגשות צד בצד                   2269
התנגשות עם עצם דומם              1866
התהפכות                          1393
פגיעה בהולך רגל                  1005
התנגשות חזית בחזית                879
החלקה                             548
אחר                               377
ירידה מהכביש או עלייה למדרכה      325
התנגשות עם רכב שנעצר ללא חניה     156
פגיעה בנוסע בתוך כלי הרכב          28
Name: accident_type, dtype: int64

# Junctions dataframe

In [16]:

df_junctions = pd.DataFrame({
    'name': ['Eilot',
        'Meches', 'Beer Ora', 'Timna', 'Samar', 'Yotvata', 'Grofit', 'Ktora', 'Yahel', 'Menuha', 'Paran',
        'Tsukim', 'Tzofar', 'Sapir', 'Ein Yahav', 'Hazeva', 'Idan', 'Haarava Junc'
        ],
    'location':[
        11.9, 15.6, 27, 36, 41.3, 49.5, 54, 60.4, 72.3, 97.6, 105.9, 120.4, 128.9, 134.8, 138.6, 153.4, 158.2, 180.5]
})

df_junctions.to_csv("../data/preprocessed/junctions.csv", index=False)

In [17]:
df_junctions

,name,location
0,Eilot,11.9
1,Meches,15.6
2,Beer Ora,27.0
3,Timna,36.0
4,Samar,41.3
5,Yotvata,49.5
6,Grofit,54.0
7,Ktora,60.4
8,Yahel,72.3
9,Menuha,97.6
